## Reddit kommentit

Pushshift API https://github.com/pushshift/api

In [ ]:
import requests 
import json
import pandas as pd

q = '' #hakutermi(stö)
subreddit='' #alareddit
after = '' #alkaen ajankohdasta (esim. 2020-06-01) tai unix timestamp
before = '' #ennen ajankohtaa


def getPushshiftData(subreddit=None, before=None, after=None, ids=None, q=None):
    suffix=''
    if (before is not None):
        suffix += '&before='+str(before)
    if (after is not None):
        suffix += '&after='+str(after)
    if (subreddit is not None):
        suffix += '&subreddit='+subreddit
    if (q is not None):
        suffix += '&q='+str(q)

    url = 'https://api.pushshift.io/reddit/search/comment/?sort=desc&size=1500'+suffix
    print('loading '+url)
    r = requests.get(url)
    data = json.loads(r.content)
    if len(data['data']) > 0:
        prev_end_date = data['data'][-1]['created_utc']
    else:
        prev_end_date = None
    return (data, prev_end_date)

(comments_tmp, prev_end_date) = getPushshiftData(subreddit=subreddit, q=q, after=after, before=before)
comments = comments_tmp['data']
while prev_end_date is not None:
    (comments_tmp, prev_end_date) = getPushshiftData(subreddit=subreddit, q=q, before=prev_end_date-1, after=after)
    if prev_end_date is not None:
        comments.extend(comments_tmp['data'])
        